<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

In [ ]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tensorflow.keras import initializers
import tensorflow as tf

#wandb.init(project="assignment-1", entity="swathi")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [ ]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [ ]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[0,j]] not in labels:
      images.append(X_train[:,j].reshape([28,28]))
      labels.append(classes[Y_train[0,j]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [ ]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
log_images()

Defining basic parameters

In [ ]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_train.shape[0]
l = len(N) - 1

Weights initialising

In [ ]:
def initialize_weights_random(N, weight_factor):
  np.random.seed(0)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append((np.random.randn(N[i],N[i-1]))*weight_factor)
    b.append(np.zeros((N[i],1)))

  return W,b 

In [ ]:
def initialize_weights_xavier(N, weight_factor):
  initializer = tf.keras.initializers.GlorotNormal()
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    values = initializer(shape=(N[i],N[i-1]))
    W.append(np.array(values)*weight_factor)
    b.append(np.zeros((N[i],1)))
    del values
  return W,b 

In [ ]:
def initialize_weights(N,init, weight_factor):
  if init=='random':
    return initialize_weights_random(N, weight_factor)

  if init=='xavier':
    return initialize_weights_xavier(N, weight_factor)

In [ ]:
def one_hot(X,y):
  one_hot_y=np.zeros((10,X.shape[1]))
  for i in range(X.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

Softmax

In [ ]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [ ]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

Sigmoid derivative

In [ ]:
def dSigmoid(a):
  return sigmoid(a)*(1-sigmoid(a))

Tanh activation function

In [ ]:
def tanh(a):
  return np.tanh(a)

Derivative of the tanh function

In [ ]:
def dTanh(a):
  return 1- np.square(tanh(a))

ReLU

In [ ]:
def ReLU(a):
  return np.maximum(0,a)

Derivative of ReLU

In [ ]:
def dReLU(a):
  a[a>=0]=1
  a[a<0]=0
  return a

In [ ]:
def activation(a,act):
  if act=='sigmoid':
    return sigmoid(a)
  if act=='tanh':
    return tanh(a)
  if act=='relu':
    return ReLU(a)

def dActivation(a,act):
  if act=='sigmoid':
    return dSigmoid(a)
  if act=='tanh':
    return dTanh(a)
  if act=='relu':
    return dReLU(a)

Cross Entropy

In [ ]:
def calculate_loss(y_hat,Y, W, alpha):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[0,i],i])
  loss = loss/Y.shape[1]
  print (loss)

In [ ]:
def dLoss(y_hat, X, y):
  return (y_hat - one_hot(X,y))

In [ ]:
def l2_loss(y_hat,Y):
  loss=(np.linalg.norm(y_hat-Y))/Y.shape[1]
  return loss

In [ ]:
def dl2loss(y_hat,Y):
  

In [ ]:
def accuracy(y_hat,Y):
  return ((y_hat == Y).sum()/Y.shape[1])

Feed forward network

In [ ]:
def feedforward(X_input, W, b, N, l, act):
  a=[]
  a.append(np.array([0]))
  h=[]
  h.append(X_input)
  for i in range(1,l):
    a_new=b[i]+np.matmul(W[i],h[i-1])
    h_new=activation(a_new,act)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [ ]:
def backprop(X, y,y_hat,W,a,h,act,alpha):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(X,y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = (np.matmul(da,np.transpose(h[i-1])) + alpha*W[i])/(X.shape[1])
    db = np.sum(da, axis=1, keepdims=True)/(X.shape[1])
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dActivation(a[i-1],act)

    grad_W.append(dw)
    grad_b.append(db)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse() 

  return grad_W,grad_b


Gradient Descent

In [ ]:
def gradient_descent(epochs, learn_rate, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init,weight_factor)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      a, h, y_hat = feedforward(X, W, b, N, l,act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        W_new.append(W[k] - (learn_rate*grad_W[k] - learn_rate*alpha*W[k]))
        b_new.append(b[k] - (learn_rate*grad_b[k]))
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = gradient_descent(10,0.1,32,'sigmoid','random',N, 0.0005,1)

1.0320659745070455
0.805644944230451
0.7250846629851776
0.6352918651591141
0.5657265080133013
0.5152962936297938
0.5022593193743711
0.4954260434252678
0.5005874589318989
0.49496262881161235


Momentum based gradient descent

In [ ]:
def mom_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init,weight_factor)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k] + learn_rate*alpha*W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new.append((W[k] - update_w[k]/batch_size))
        b_new.append((b[k] - update_b[k]/batch_size))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = mom_grad_descent(10,0.1,0.9,32,'sigmoid','random', N, 0.0005, 1)

1.350061796084539
1.0889110488170297
0.9839705003429449
0.8766992278073104
0.7917367114492417
0.7287367977756134
0.6821837421198577
0.6457019750906711
0.6167947806390108
0.5901190681460945


Nesterov accelerated gradient descent

In [ ]:
def nesterov_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init, weight_factor)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k]
        update_b[k] = gamma*update_prev_b[k]
        W_new.append((W[k] - update_w[k]/batch_size))
        b_new.append((b[k] - update_b[k]/batch_size))
      W = W_new
      b = b_new
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      for k in range(1,l+1):
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k] + learn_rate*alpha*W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new[k] = (W[k] - update_w[k]/batch_size)
        b_new[k] = (b[k] - update_b[k]/batch_size)
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = nesterov_grad_descent(10,0.1,0.9,32,'sigmoid','random',N, 0.0005, 1)

1.155724441241262
0.9145060774612552
0.7537736252764919
0.6708991415471723
0.6151469692213979
0.5695331653622396
0.5382136492905488
0.5120454651703036
0.4867283173238386
0.4609558239105257


RMS Prop

In [ ]:
def rms_prop(epochs, learn_rate, beta, epsilon, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init, weight_factor)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = beta*update_prev_w[k] + (1-beta)*np.square(grad_W[k])
        #print (update_w[k][0][0])
        update_b[k] = beta*update_prev_b[k] + (1-beta)*np.square(grad_b[k])
        W_new.append((W[k] - (learn_rate*np.divide(grad_W[k],(np.sqrt(update_w[k]+epsilon)))) - learn_rate*alpha*W[k]))
        b_new.append((b[k] - (learn_rate*np.divide(grad_b[k],(np.sqrt(update_b[k]+epsilon))))))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = rms_prop(10,0.001,0.9,10**-8,16,'tanh','random',N, 0.005, 1)

ZeroDivisionError: ignored

In [ ]:
def adam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init, weight_factor)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        t = i*n + j+1
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon)))) - learn_rate*alpha*W[k]))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = adam(10,0.01,0.9,0.99, 10**-8,32,'sigmoid','random',N, 0.005, 1)

2.980883245203898
2.8776820848736793
2.7882420326710635
2.711105477785292
2.64499697402725
2.5887549675562362
2.541290213508981
2.4995732201538425
0.9615357649140022
0.4381751429952531


In [ ]:
def nadam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N, alpha, weight_factor):
  W, b = initialize_weights(N,init, weight_factor)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        t = i*n + j+1
        m_w[k] = (beta1*m_prev_w[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k])/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k])/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
      W = W_new
      b = b_new
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      for k in range(1,l+1):
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new[k] = W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon)))) - learn_rate*alpha*W[k]
        b_new[k] = b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
    y_hat = feedforward(X_train, W, b, N, l,act)[2]
    loss=calculate_loss(y_hat, Y_train)
    acc=accuracy(y_hat,Y_train)
    y_hat_val = feedforward(X_validation, W, b, N, l,act)[2]
    val_loss=calculate_loss(y_hat_val, Y_validation)
    val_acc=accuracy(y_hat_val,Y_validation)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = nadam(10, 0.1,0.9,0.99,10**-8,32,'sigmoid','random',N, 0.005, 1)

2.9739171451630675
2.6202837304219
2.4920760418688763
2.44359896450905
2.4241837636084367
2.4160883456202242
2.4126454405165463
2.3857659675163396
1.7030305189521646


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


1.9668050508560238


In [ ]:
def predict(W,b,X_test,N,l, act):
  y_hat = feedforward(X_test, W, b, N, l, act)[2]
  y_class = np.argmax(y_hat,axis=0).reshape([1,X_test.shape[1]])

  return y_class

In [ ]:
import sklearn.metrics
Y_predict = predict(W,b,X_validation,N,l,'tanh')
correct = 0
for i in range(Y_validation.shape[1]):
  if Y_predict[0][i]==Y_validation[0][i]:
    correct +=1
correct = correct/Y_validation.shape[1]
print (correct)

0.7776666666666666


In [ ]:
sweep_config = {
    'method': 'grid'
    }

In [ ]:
parameters_dict = {
    # 'l': {
    #     'values': [3, 4, 5]
    #   },
    # 'size of each layer': {
    #     'values': [32, 64, 128]
    #   },
    'weight_factor': {
        'values': [0, 0.0005, 0.5]
      },
    'learn_rate': {
        'values': [1e-3,1e-4]
      },
    'epochs': {
        'values': [5,10]
      },
    'batch_size': {
        'values': [16,32,64]
      },
    'init': {
        'values': ['random', 'xavier']
      },
    'optimizer': {
        'values': ['sgd', 'mom', 'nest', 'rms', 'adam', 'nadam']
      },
    'act': {
          'values': ['sigmoid', 'tanh', 'ReLU']
      }
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg1")

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
def training_sweep(config=None):
  
  # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        if config.optimizer == 'sgd':
          gradient_descent(config.epochs, config.learn_rate, config.batch_size, config.act, config.init, N, config.weight_factor)
        if config.optimizer == 'mom':
          mom_grad_descent(config.epochs, config.learn_rate, gamma=0.9, config.batch_size, config.act, config.init, N, config.weight_factor)
        if config.optimizer == 'nest':
          nesterov_grad_descent(config.epochs, config.learn_rate, gamma=0.9, config.batch_size, config.act, config.init, N, config.weight_factor)
        if config.optimizer == 'rms':
          rms_prop(config.epochs, config.learn_rate, beta=0.9, epsilon=10**-8,config.batch_size, config.act, config.init, N, config.weight_factor)
        if config.optimizer == 'adam':
          adam(config.epochs, config.learn_rate, beta1=0.9, beta2=0.99, epsilon=10**-8,config.batch_size, config.act, config.init, N, config.weight_factor)
        if config.optimizer == 'nadam':
          nadam(config.epochs, config.learn_rate, beta1=0.9, beta2=0.99, epsilon=10**-8,config.batch_size, config.act, config.init, N, config.weight_factor)

        #wandb.log({"loss": avg_loss, "epoch": epoch})